In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd 'drive/MyDrive/Tensorflow_2'

Mounted at /content/drive
/content/drive/MyDrive/Tensorflow_2


In [ ]:
import functions as f

from Text import *
from LSTM_class import *
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras import layers, models, optimizers

In [ ]:
path_train = '/content/drive/MyDrive/Tensorflow_2/merged_clean.txt'

input_train = f.read_txt(path_train)

In [ ]:
max_len = 4
step = 3

text_train = Text(input_train)
text_train.tokens_info()

seq_train = Sequences(text_train, max_len, step)
seq_train.sequences_info()

total tokens: 2734890, distinct tokens: 41044
number of sequences of length 4: 911629


In [ ]:
print(text_train.tokens[:10])
print(text_train.tokens_ind[:10], '\n')

np.array(seq_train.sequences[:2])

['Lovely', 'IlonkaThere', 'was', 'once', 'a', "king's", 'son', 'who', 'told', 'his']
[32030, 36371, 5124, 16854, 24911, 28905, 14370, 1863, 21176, 15055] 



array([[32030, 36371,  5124, 16854],
       [16854, 24911, 28905, 14370]])

In [ ]:
batch_size = 4096

params = {
  'sequence_length': max_len,
  'vocab_size': len(text_train),
  'batch_size': batch_size,
  'shuffle': True
}

train_generator = TextDataGenerator(seq_train.sequences, seq_train.next_words, **params)

In [ ]:
def lstm_model(sequence_length, vocab_size, layer_size, embedding=False):
  model = models.Sequential()
  if embedding:
    model.add(layers.Embedding(vocab_size, layer_size))
    model.add(layers.LSTM(layer_size))    
  else:
    model.add(layers.LSTM(layer_size, input_shape=(sequence_length, vocab_size)))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(vocab_size, activation='softmax'))
  return model

In [ ]:
model = lstm_model(max_len, len(text_train), 512)

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=2,
          verbose=1)

Epoch 1/2
222/222 [==============================] - 593s 3s/step - loss: 5.9282
Epoch 2/2
183/222 [=======================>......] - ETA: 1:43 - loss: 4.9188

In [ ]:
model.save('data/out/lstm_model')

#Text generation with LSTM model

In [ ]:
token2ind, ind2token = text_train.token2ind, text_train.ind2token

input_prefix = 'Once upon a time'
text_prefix = Text(input_prefix, token2ind, ind2token)

In [ ]:
pred = ModelPredict(model, text_prefix, token2ind, ind2token, max_len)

In [ ]:
temperatures = [1, 0.7, 0.4, 0.1]

for temperature in temperatures:
  print('temperature:', temperature)
  print(pred.generate_sequence(100, temperature=0.7))
  print('\n')

#Text generation with LSTM model with Embedding layer

In [ ]:
batch_size_emb = 4096

params_emb = params.copy()
params_emb['embedding'] = True

train_generator_emb = TextDataGenerator(seq_train.sequences, seq_train.next_words, **params_emb)

In [ ]:
model_emb = lstm_model(max_len, len(text_train), 512, embedding=True)
model_emb.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
model_emb.fit(train_generator_emb,
              steps_per_epoch=len(train_generator_emb),
              epochs=40,
              verbose=1)

Epoch 1/40
222/222 [==============================] - 113s 508ms/step - loss: 5.2527
Epoch 2/40
222/222 [==============================] - 112s 503ms/step - loss: 4.8233
Epoch 3/40
222/222 [==============================] - 113s 505ms/step - loss: 4.6237
Epoch 4/40
222/222 [==============================] - 112s 504ms/step - loss: 4.4655
Epoch 5/40
222/222 [==============================] - 113s 506ms/step - loss: 4.3315
Epoch 6/40
222/222 [==============================] - 115s 516ms/step - loss: 4.2182
Epoch 7/40
222/222 [==============================] - 115s 516ms/step - loss: 4.1238
Epoch 8/40
222/222 [==============================] - 115s 517ms/step - loss: 4.0378
Epoch 9/40
222/222 [==============================] - 115s 517ms/step - loss: 3.9649
Epoch 10/40
222/222 [==============================] - 115s 515ms/step - loss: 3.9033
Epoch 11/40
222/222 [==============================] - 114s 514ms/step - loss: 3.8513
Epoch 12/40
222/222 [==============================] - 115s 517

In [ ]:

model_emb.save('data/out/lstm_model_emb')

In [ ]:
pred_emb = ModelPredict(model_emb, text_prefix, token2ind, ind2token, max_len, embedding=True)

In [ ]:
temperatures = [1, 0.7, 0.4, 0.1]

for temperature in temperatures:
  print('temperature:', temperature)
  print(pred_emb.generate_sequence(100, temperature=0.7))
  print('\n')